In [ ]:
import time
import threading
import random
import os
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.tag_value import TagValue
from datetime import datetime
# New imports for checking trading days
import pandas_market_calendars as mcal
from pandas.tseries.offsets import BDay

TOTAL_SHARES_TO_BUY = 1500
NUMBER_OF_DAYS = 5
STOCK_SYMBOL = "NVDA"
ACTION = "SELL"
LMT_PRICE = 185.00

HOST = "127.0.0.1"
PORT = 7497

PROGRESS_FILE = "trade_progress.txt"

class PlaceOrderApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.next_order_id = None
        self.order_id_received = threading.Event()
        self.order_filled_event = threading.Event()

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.next_order_id = orderId
        print(f"Next valid order ID: {orderId}")
        self.order_id_received.set()

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice,
                    permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. ID: {orderId}, Status: {status}, Filled: {filled}, "
              f"Remaining: {remaining}, AvgFillPrice: {avgFillPrice}")
        if remaining == 0:
            print("Order has been completely filled.")
            self.order_filled_event.set()

    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson=None):
        if errorCode in [2104, 2106, 2158, 202, 399]:
            return
        print(f"Error. ID: {reqId}, Code: {errorCode}, Msg: {errorString}")

class MultiDayTrader:
    def __init__(self, total_quantity, num_days, symbol):
        self.total_quantity = total_quantity
        self.num_days = num_days
        self.symbol = symbol
        self.quantities_per_day = self._calculate_daily_quantities()
        self.lmt_price_cap = LMT_PRICE

    def _calculate_daily_quantities(self):
        if self.num_days <= 0:
            return []
        base_qty = self.total_quantity // self.num_days
        remainder = self.total_quantity % self.num_days
        quantities = [base_qty] * self.num_days
        for i in range(remainder):
            quantities[i] += 1
        return quantities

    def get_next_trading_day(self):
        """Read progress file to get day index. Returns None if all days are done."""
        day_index = 0
        if os.path.exists(PROGRESS_FILE):
            with open(PROGRESS_FILE, "r") as f:
                content = f.read().strip()
                if content.isdigit():
                    day_index = int(content)
        if day_index >= self.num_days:
            return None
        return day_index

    def update_progress(self, day_index):
        with open(PROGRESS_FILE, "w") as f:
            f.write(str(day_index + 1))

    def run_trading_session(self, day, daily_quantity):
        app = PlaceOrderApp()
        print(f"\n--- Day {day + 1} of {self.num_days} ---")
        print(f"Attempting to {ACTION} {daily_quantity} shares of {self.symbol}.")

        client_id = random.randint(1, 100000)
        print(f"Connecting with Client ID: {client_id}...")
        app.connect(HOST, PORT, clientId=client_id)
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()

        if not app.order_id_received.wait(timeout=30):
            print("Failed to get a valid order ID from TWS within 30 seconds. Aborting today's session.")
            app.disconnect()
            return

        contract = Contract()
        contract.symbol = self.symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        
        order = Order()
        order.action = ACTION
        order.orderType = "LMT"
        order.totalQuantity = daily_quantity
        order.lmtPrice = self.lmt_price_cap
        order.tif = "DAY"
        order.algoStrategy = "Twap"
        today = datetime.now().strftime("%Y%m%d")
        order.algoParams = [
            TagValue("startTime", f"{today} 10:00:00 US/Eastern"),
            TagValue("endTime", f"{today} 15:45:00 US/Eastern")
        ]

        app.placeOrder(app.next_order_id, contract, order)
        print("Daily TWAP order has been placed.")

        print("Waiting for order to be filled...")
        filled = app.order_filled_event.wait(timeout=30)
        if filled:
            print("Today's order confirmed filled.")
        else:
            print("Order not confirmed as filled within the timeout. Check TWS for status.")

        print("Disconnecting for the day.")
        app.disconnect()
        time.sleep(2)

    def trade_today(self):
        next_day = self.get_next_trading_day()
        if next_day is None:
            print("All trading days are complete. No action taken.")
            return

        daily_quantity = self.quantities_per_day[next_day]
        if daily_quantity <= 0:
            print(f"No shares to trade for day {next_day + 1}. Skipping.")
        else:
            self.run_trading_session(next_day, daily_quantity)
        self.update_progress(next_day)
        if next_day + 1 >= self.num_days:
            print("\nMulti-day trading strategy complete.")

def is_market_open_today():
    """Checks if today is a trading day on the NYSE."""
    nyse = mcal.get_calendar('NYSE')
    today = datetime.now().date()
    # Get the schedule for today. It will be empty if it's not a trading day.
    schedule = nyse.schedule(start_date=today, end_date=today)
    return not schedule.empty

if __name__ == "__main__":
    # --- NEW LOGIC ---
    # First, check if the market is open today
    if not is_market_open_today():
        print(f"Today ({datetime.now().strftime('%Y-%m-%d')}) is not a trading day. Exiting.")
    else:
        print(f"Today ({datetime.now().strftime('%Y-%m-%d')}) is a trading day. Proceeding with script.")
        trader = MultiDayTrader(
            total_quantity=TOTAL_SHARES_TO_BUY,
            num_days=NUMBER_OF_DAYS,
            symbol=STOCK_SYMBOL,
        )
        trader.trade_today()
